In [529]:
import numpy as np
from PIL import Image
import re
from css_parser import *
from fpdf import FPDF
from color_utils import *
import os.path

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [530]:
#http://www.fpdf.org/en/script/script74.php
class AlphaPDF (FPDF):
    def __init__ (self, orientation = 'P', unit = 'mm', format = 'A4'):
        self.__extgstates = []
        super().__init__ (orientation, unit, format)
        self.bm = 'Normal'
        self.alpha = 1
    # alpha: real value from 0 (transparent) to 1 (opaque)
    # bm:    blend mode, one of the following:
    #          Normal, Multiply, Screen, Overlay, Darken, Lighten, ColorDodge, ColorBurn,
    #          HardLight, SoftLight, Difference, Exclusion, Hue, Saturation, Color, Luminosity
    def set_alpha(self, alpha, bm='Normal'):
        # set alpha for stroking (CA) and non-stroking (ca) operations
        self.bm = bm
        self.alpha = alpha

        data = {'ca':alpha,'CA':alpha,'BM':'/' + bm, 'n' : 0}
        gs = self.add_ext_gstate(data)
        self.set_ext_gstate(gs + 1)

    def add_page(self, orientation=''):
        # do normal add page stuff
        super().add_page(orientation)
        self.set_alpha(self.alpha, self.bm)
        

    def add_ext_gstate(self, data):
        n = len(self.__extgstates)
        self.__extgstates.append(data)
        return n

    def _enddoc(self):
        if len(self.__extgstates) > 0 and self.pdf_version < '1.4':
            self.pdf_version ='1.4'
        super()._enddoc()

    def set_ext_gstate(self, gs):
        self._out('/GS%d gs' % (gs))

    def _putextgstates(self):
        i = 0
        while i < len(self.__extgstates):
            self._newobj()
            self._out('<</Type /ExtGState')
            self.__extgstates[i]["n"] = self.n
            parms = self.__extgstates[i]
            self._out('/ca %.3F' % (parms["ca"]))
            self._out('/CA %.3F' % (parms["CA"]))
            self._out('/BM ' + parms["BM"])
            self._out('>>')
            self._out('endobj')
            i += 1

    def _putresourcedict(self):
        super()._putresourcedict()
        self._out('/ExtGState <<')
        for index, eg in enumerate(self.__extgstates):
            self._out('/GS' + str(index + 1) + ' ' + str(eg["n"])  + ' 0 R')
        self._out('>>')

    def _putresources(self):
        self._putextgstates()
        super()._putresources()
        
class ColoredPDF (AlphaPDF):
    
    def __init__(self, orientation='P',unit='mm',format='A4'):
        super().__init__(orientation, unit, format)
        self.background_color = ""

    #hex color
    def set_background_color(self, color):
        if type(color) in (tuple, list, np.array) and (len(color) == 3 or len(color) == 4):
            color = rgb_to_hex(color)

        self.background_color = color
        
    def set_fill_color_str(self, fill_color):
        self.fill_color = fill_color
        self.color_flag=(self.fill_color!=self.text_color)
        if(self.page>0):
            self._out(self.fill_color)

    def add_page(self, orientation=''):
        # do normal add page stuff
        super().add_page(orientation)

        if self.background_color:
            # before pdf did it's content placement stuff insert background
            rgb = hex_to_rgba(self.background_color)

            last_alpha = self.alpha
            last_bm = self.bm
            last_fill = self.fill_color

            self.set_alpha(1)

            self.set_fill_color(rgb[0], rgb[1], rgb[2])
            self.rect(0,0, 210, 297, "F")

            self.set_alpha(last_alpha, last_bm)
            self.set_fill_color_str(last_fill)

In [883]:
def try_convert_color(color):
    if type(color) is str:
        color = hex_to_rgba(color)
    return color
    
class TextStyle:
    def __init__(self, font_size = 16, font_color = [5, 5, 5], background_color = None):
        self.font_size = font_size
        
        self.font_color = try_convert_color(font_color)
        self.background_color = try_convert_color(background_color) 

class DocStyle:
    def __init__(self, background_color = [220, 220, 220]):
        self.background_color = background_color


class StyledPDF(ColoredPDF):
    def __init__(self, orientation='P',unit='mm',format='A4'):
        super().__init__(orientation, unit, format)

        self.set_doc_style(DocStyle())

        self.set_text_style(TextStyle())

    def set_doc_style(self, style: DocStyle):
        self.doc_style = style
        self.set_background_color(style.background_color)
        
    def set_text_style(self, style: TextStyle):
        self.text_style = style

        self.set_font("Arial", size = style.font_size)
        self.set_text_color(style.font_color[0], style.font_color[1], style.font_color[2])

        if style.background_color is not None:
            self.set_fill_color(style.background_color[0], style.background_color[1], style.background_color[2])
        else:
            color = self.doc_style.background_color
            self.set_fill_color(color[0], color[1], color[2])

class PrintablePDF(StyledPDF):
    def __init__(self, orientation='P',unit='mm',format='A4'):
        self.text_buffer = ""
        self.cell_h = 10
        self.cell_w = 190

        self.pointer_x = 0
        self.pointer_y = 0
        
        super().__init__(orientation, unit, format)

    def bullet(self):
        self.cell(1, 7, '.', align =  'R')

    def print_char(self, x):
        self.text_buffer += x

        if x == '\n':
            self.finish_print()

            self.pointer_x = self.l_margin
            self.pointer_y += self.cell_h
            self.set_xy(self.pointer_x, self.pointer_y)            
            
    def finish_print(self):
        if not self.text_buffer:
            return
        
        ret = self.multi_cell(0, self.cell_h, self.text_buffer, split_only = True)
        
        fill = self.text_style.background_color is not None
        
        if len(ret) > 1:
            r = ret[0]
            self.cell(self.cell_w, self.cell_h, r, 0, 2, 'J', fill)

            self.set_x(self.l_margin)
            
            remaining_parts = ret[1:]
            text = " ".join(remaining_parts)
                
            ret = self.multi_cell(0, self.cell_h, text, split_only = True)                
        
        self.print_bloks(ret)
        

        self.text_buffer = ""
    
    def print_bloks(self, ret):
        fill = self.text_style.background_color is not None

        for r in ret:
            chars = self.get_string_width(r)# np.add.reduce([cw.get(c,0) for c in r])
            
            self.pointer_x = chars+ self.x# + self.l_margin 
            self.pointer_y = self.y

            self.cell(self.cell_w, self.cell_h, r, 0, 2, 'J', fill)

    def set_text_style(self, style: TextStyle):
        if self.text_buffer:
            self.finish_print()
            self.set_xy(self.pointer_x, self.pointer_y)
        
        super().set_text_style(style)
        
#          |
#dasada d d
# s BB   
        

In [899]:
input_file = "MDtest.md"
file = open(input_file, "r")
file_text = file.read()
file.close()

In [900]:
style_normal = TextStyle(14, "#ff33cc")
style_normal2 = TextStyle(14, "#ff5050")

style_h1 = TextStyle(24, "#ff33cc")
style_h2 = TextStyle(22, "#33cccc")
style_h3 = TextStyle(20, "#cc6600")
style_h4 = TextStyle(18, "#0000ff")
style_h5 = TextStyle(16, "#660066")
style_h5 = TextStyle(14, "#ff33cc")

styles_headers = [style_h1, style_h2, style_h3, style_h4, style_h5]

In [914]:
lexems = ["#", "##", "###", "####", "#####", "######"]
lex_to_style = {
    lexems[0]: style_h1,
    lexems[1]: style_h2,
    lexems[2]: style_h3,
    lexems[3]: style_h4,
    lexems[4]: style_h5,
}

In [917]:
pdf = PrintablePDF()

pdf.add_page()

#pdf.set_text_style(style_h3)

new_line = True
current_word = ""
counter = 0

for x in file_text:
    if x == '\n':
        new_line = True
        counter = 0
    else:
        pass

    if counter > 0:
        new_line = False

    if x == ' ':
        current_word = ""
    else:
        current_word += x
        print("add ", x)
    print(current_word, len(current_word))

    if current_word in lex_to_style:
        pdf.set_text_style(lex_to_style[current_word])
    
    pdf.print_char(x)
    
    counter += 1

pdf.finish_print()
   
pdf.output("mygfg.pdf")   

add  #
# 1
 0
add  T
T 1
add  i
Ti 2
add  t
Tit 3
add  l
Titl 4
add  e
Title 5
 0
add  1
1 1
add  

1
 2
add  #
1
# 3
add  #
1
## 4
 0
add  T
T 1
add  i
Ti 2
add  t
Tit 3
add  l
Titl 4
add  e
Title 5
 0
add  2
2 1
add  

2
 2
add  #
2
# 3
add  #
2
## 4
add  #
2
### 5
 0
add  T
T 1
add  i
Ti 2
add  t
Tit 3
add  l
Titl 4
add  e
Title 5
 0
add  3
3 1
add  

3
 2
add  #
3
# 3
add  #
3
## 4
add  #
3
### 5
add  #
3
#### 6
 0
add  T
T 1
add  i
Ti 2
add  t
Tit 3
add  l
Titl 4
add  e
Title 5
 0
add  4
4 1
add  

4
 2
add  #
4
# 3
add  #
4
## 4
add  #
4
### 5
add  #
4
#### 6
add  #
4
##### 7
 0
add  T
T 1
add  i
Ti 2
add  t
Tit 3
add  l
Titl 4
add  e
Title 5
 0
add  5
5 1
add  

5
 2
add  #
5
# 3
add  #
5
## 4
add  #
5
### 5
add  #
5
#### 6
add  #
5
##### 7
add  #
5
###### 8
 0
add  T
T 1
add  i
Ti 2
add  t
Tit 3
add  l
Titl 4
add  e
Title 5
 0
add  6
6 1
 0
add  +
+ 1
 0
add  1
1 1
add  

1
 2
add  

1

 3
add  >
1

> 4
add  h
1

>h 5
add  i
1

>hi 6
add  g
1

>hig 7
add  h
1

>high 8
add  l
1


''